In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [3]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [18]:
driver = webdriver.Chrome(driver_name)

driver.get(url)
time.sleep(2)

n_page = 0
# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

end, current = len(lis), 0
while True:

    try:
        trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    except:
        print('nPgae = ', n_page)
        break

    lines = []
    for tr in trs:

        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(2)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > p').text
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')

        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')

        # 코드 정보
        code = dds[0].text
        summering = dds[3].text
        # 개봉일
        first_day = dds[5].text
        # 제작년도
        production_year = dds[6].text
    
        
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div > dd')
        movie_director, star_actor = '', ''
        if len(tmp) >= 1:
            movie_director = tmp[0].text
        if len(tmp) >= 2:
            tmp2 = tmp[1].find_elements(By.TAG_NAME, 'table')
            
            if len(tmp2) >= 1:
                star_actor = tmp2[0].text
            else:
                star_actor = tmp[1].text
            star_actor = star_actor.replace('[주연] ', '')
            # print(star_actor)

        lines.append({'code': code, 'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})

        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
        time.sleep(1)


    current += 1
    n_page += 1

    if n_page == 100: break

    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(2)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        lis[current].click()
        time.sleep(3)

df = pd.DataFrame(lines)

df.to_csv('../static/data/test.csv', index=False)

In [21]:
df = pd.DataFrame(lines)


In [ ]:
driver.close()

In [22]:
df

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20183358,겨울밤에 (Winter's Night),드라마,"장편 | 예술,독립 영화 | 드라마 | 91분 43초 | 12세이상관람가 | 한국",2020-12-10,2018년,장우진,서영화(은주) | 양흥주(흥주) | 이상희(여자) | 우지현(남자) | 김선영(해란),https://www.kobis.or.kr/common/mast/movie/2020...,중년의 부부 은주와 흥주가 30년 만에 춘천의 청평사를 방문한다. 그곳을 빠져나오는...
1,20174602,노무현입니다 (OUR PRESIDENT),다큐멘터리,장편 | 일반영화 | 다큐멘터리 | 109분 7초 | 12세이상관람가 | 한국,2017-05-25,2017년,이창재,노무현,https://www.kobis.or.kr/common/mast/movie/2017...,"국회의원, 시장 선거 등 출마하는 선거마다 번번이 낙선했던 만년 꼴찌 후보 노무현이..."
2,20223585,들랑날랑 혼삿길 (I Smell Wedding Bells),다큐멘터리,장편 | 일반영화 | 다큐멘터리 | 41분 0초 | 한국,해당정보없음,2021년,홍민키,,https://www.kobis.or.kr/kobis/web/comm/images/...,‘혼삿길에 우뚝 선 오픈리-게이’ 성소수자인 주인공 민기는 결혼 적령기에 들어감에 ...
